In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)
%cd '/content/drive/Shared drives/MinneMUDAC/Data Challenge details'

Mounted at /content/drive/
/content/drive/Shared drives/MinneMUDAC/Data Challenge details


In [3]:
def convert_team_ids(team):
    if (team == 'LAA'):
        return 'ANA' # matches up with previous table
    if (team == 'FLO'):
        return 'MIA' # matches up with previous table
    if (team == 'MON'):
        return 'WAS' # matches up with previous table
    return team

In [5]:
g = pd.read_csv('./data/GameLogs.csv', encoding='utf-16')
g.iloc[:, 0]=pd.to_datetime(g.iloc[:,0], format = '%Y%m%d')
g.HomeTeam = g.HomeTeam.apply(convert_team_ids)
g.VisitingTeam = g.VisitingTeam.apply(convert_team_ids)

In [ ]:
username = "geonode_cySpFcf4fD"
password = "0a9305dd-b60f-440d-adad-17aebe1a4c22"
GEONODE_DNS = "premium-residential.geonode.com:9000"
proxy = {"http":"http://{}:{}@{}".format(username, password, GEONODE_DNS)}

In [6]:
teams = g['HomeTeam'].unique()

In [7]:
def get_batting():
    team_df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
                                      'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
                                      'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB'])
    for team in teams:
        #print(team)
        #print(team_df.head())
        team_df = team_df.append(pd.Series(team, index=team_df.columns), ignore_index=True)
        for year in range (2000, 2023):
            try:
                team_df = team_df.append(pd.Series(year, index=team_df.columns), ignore_index=True)
                df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
                                             'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
                                             'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB'])
                url = 'http://www.baseball-reference.com/teams/' + team + '/' + str(year) + '.shtml'
                html = requests.get(url, proxies=proxy)
                bs = BeautifulSoup(html.text, 'html.parser')
                table = bs.find('table', {'id':'team_batting'})
                for game in table.find_all('tr'):
                    results = []
                    for element in game.find_all('td'):
                        results.append(element.text)
                        if len(results) == 27:
                            df = df.append(pd.Series(results, index=df.columns), ignore_index=True)

            # create one major df
                team_df = pd.concat([team_df, df]).reset_index(drop=True)

            # sleep
                sleep(3)
              
            except:
                pass

    return team_df

In [8]:
%time all_teams = get_batting()
all_teams.to_csv('./data/mlb-batting-data-2000-2022.csv', index=False, encoding='utf-8')

CPU times: user 2.24 s, sys: 54.9 ms, total: 2.3 s
Wall time: 2.2 s


In [ ]:
def get_pitching():
    team_df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 
                                      'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO',
                                      'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9',
                                      'BB9', 'SO9', 'SO/W'])
    for team in teams:
        #print(team)
        #print(team_df.head())
        team_df = team_df.append(pd.Series(team, index=team_df.columns), ignore_index=True)
        for year in range (2000, 2023):
            try:
                team_df = team_df.append(pd.Series(year, index=team_df.columns), ignore_index=True)
                df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 
                                      'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO',
                                      'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9',
                                      'BB9', 'SO9', 'SO/W'])
                url = 'http://www.baseball-reference.com/teams/' + team + '/' + str(year) + '.shtml'
                html = requests.get(url, proxies=proxy)
                bs = BeautifulSoup(html.text, 'html.parser')
                table = bs.find('table', {'id':'team_pitching'})
                for game in table.find_all('tr'):
                    results = []
                    for element in game.find_all('td'):
                        results.append(element.text)
                        if len(results) == 33:
                            df = df.append(pd.Series(results, index=df.columns), ignore_index=True)

            # create one major df
                team_df = pd.concat([team_df, df]).reset_index(drop=True)

            # sleep
                sleep(3)
              
            except:
                pass

    return team_df

In [ ]:
%time all_teams = get_pitching()
all_teams.to_csv('./data/mlb-pitching-data-2000-2022.csv', index=False, encoding='utf-8')